# Evaluate BLIP model offline

Let’s start by loading our trained model and our test data.

In [9]:
#imports
import os
import base64
import torch
import pandas as pd
import numpy as np
import random
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import InterpolationMode
from torch.utils.data import DataLoader, Dataset, Subset
from torchsummary import summary
from sklearn.model_selection import StratifiedKFold

In [12]:
# triton_server_url = "http://localhost:8010/v2/models/caption/infer"
triton_server_url = "http://localhost:8001"

In [13]:
def test_data():
    # Replace with your actual data pipeline
    data = [
        {"image_path": "/Users/manali/nyu/COURSES/Sem4/MLOps/ECE-GY-9183-Machine-Learning-Systems-Engg-Operations-Project/src/serving/tests/captioning_dummy_test_images/10002456.jpg", "expected_caption": "Workers look down from up above on a piece of equipment ."},
        {"image_path": "/Users/manali/nyu/COURSES/Sem4/MLOps/ECE-GY-9183-Machine-Learning-Systems-Engg-Operations-Project/src/serving/tests/captioning_dummy_test_images/1000092795.jpg", "expected_caption": "Two men in green shirts are standing in a yard ."},
    ]                                                                                                                                                                       
    return data
data=test_data()
data

[{'image_path': '/Users/manali/nyu/COURSES/Sem4/MLOps/ECE-GY-9183-Machine-Learning-Systems-Engg-Operations-Project/src/serving/tests/captioning_dummy_test_images/10002456.jpg',
  'expected_caption': 'Workers look down from up above on a piece of equipment .'},
 {'image_path': '/Users/manali/nyu/COURSES/Sem4/MLOps/ECE-GY-9183-Machine-Learning-Systems-Engg-Operations-Project/src/serving/tests/captioning_dummy_test_images/1000092795.jpg',
  'expected_caption': 'Two men in green shirts are standing in a yard .'}]

In [2]:
def generate_caption_triton(image_path, triton_server_url):
    with open(image_path, 'rb') as img_file:
        img_bytes = img_file.read()

    img_base64 = base64.b64encode(img_bytes).decode('utf-8')

    payload = {
        "inputs": [
            {
                "name": "image",
                "shape": [1],
                "datatype": "BYTES",
                "data": [img_base64]
            }
        ]
    }
    response = requests.post(triton_server_url, json=payload)
    response.raise_for_status()
    result = response.json()
    caption = result['outputs'][0]['data'][0]
    return caption

In [3]:
generate_caption_triton("/Users/manali/nyu/COURSES/Sem4/MLOps/ECE-GY-9183-Machine-Learning-Systems-Engg-Operations-Project/src/serving/tests/captioning_dummy_test_images/10002456.jpg",triton_server_url)

NameError: name 'triton_server_url' is not defined

In [ ]:
def test_bleu_score_above_threshold(test_data):
    threshold = 0.3  # Adjust based on your domain needs
    for sample in test_data:
        generated = generate_caption_triton(sample['image_path'])
        reference = [sample['expected_caption'].split()]
        candidate = generated.split()
        score = sentence_bleu(reference, candidate)
        print(f"BLEU score for {sample['image_path']}: {score}")
        assert score >= threshold